**Task 07: Querying RDF(s)**

In [9]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [10]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Ncfa66e382fdd45a9aeab87bbc8dbe078 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [17]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
qP = prepareQuery('''
  SELECT ?sub
  WHERE { 
    ?sub rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)
#SPARQL
for r in g.query(qP):
  print(r)
#RDFLib
for sub,pr,obj in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(sub,pr,obj)  

(rdflib.term.URIRef('http://somewhere#Researcher'),)
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [20]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
myQuery = prepareQuery('''
  SELECT ?ind
  WHERE { 
    { 
      ?ind rdf:type ns:Person. 
    } UNION {
      ?sub rdfs:subClassOf ns:Person.
      ?ind rdf:type ?sub
    }
  }
  ''',
  initNs = { "rdf": RDF,"rdfs": RDFS, "ns": ns}
)
#SPARQL
for r in g.query(qP):
  print(r)

#RDFLib

#Individuals of Person
for sub,pr,obj in g.triples((None, RDF.type, ns.Person)):
  print(sub,pr,obj) #Give me all individuals of Person
#SubClasses of Person
for sub,pr,obj in g.triples((None,RDFS.subClassOf,ns.Person)):
  for sub2,pr2,obj2 in g.triples((None,RDF.type,sub)):
    print(sub2) #Give me all the individuals that are subClasses of Person

(rdflib.term.URIRef('http://somewhere#Researcher'),)
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [13]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
qP = prepareQuery('''
  SELECT ?ind ?pr ?class
  WHERE { 
    { 
      ?ind rdf:type ns:Person.
      ?ind ?pr ?class
    } UNION {
      ?ind rdf:type ?subC.
      ?ind ?pr ?class.
      ?subC rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "rdf": RDF,"rdfs": RDFS, "ns": ns}
)

#SPARQL
for r in g.query(qP):
  print(r)


#RDFLib
for suj,pr,obj in g.triples((None, RDFS.subClassOf, ns.Person)):
  for suj2,pr2,obj2 in g.triples((None, RDF.type, suj)):
    for suj3,pr3,obj3 in g.triples((suj2, None, None)):
      print(suj3,pr3,obj3) #Give me all subclasses of Person

for suj,pr,obj in g.triples((None, RDF.type, ns.Person)):
  for suj2,pr2,obj2 in g.triples((s, None, None)):
      print(s2,pr2,obj2) #Give me all individuals of Person


(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('Sara Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/